## Imports

In [13]:
import csv
import os
import random
import zipfile

from pathlib import Path 
from functools import partial
from typing import Callable, Any,List

import numpy as np 
import torch

from tqdm.notebook import tqdm
from data import get_challenge_points
from metrics import get_tpr_at_fpr

In [14]:
import json

from midst_models.single_table_TabDDPM.wb_complex_pipeline import (
    clava_clustering,
    clava_training,
    clava_load_pretrained,
    clava_synthesizing,
    load_configs,
)
from midst_models.single_table_TabDDPM.wb_pipeline_modules import load_multi_table
from midst_models.single_table_TabDDPM.tab_ddpm.gaussian_multinomial_diffsuion import GaussianMultinomialDiffusion

In [15]:
TABDDPM_DATA_DIR = "tabddpm_white_box"
TABSYN_DATA_DIR = "tabsyn_white_box"

## Loading the Model

In [16]:
# Load config
config_path = "/home/vidit/Desktop/SaTML/MIDSTModels/starter_kits/tabddpm_white_box/train/tabddpm_1/trans.json"
configs, save_dir = load_configs(config_path)

# Display config
json_str = json.dumps(configs, indent=4)
print(json_str)

{
    "general": {
        "data_dir": "/home/vidit/Desktop/SaTML/MIDSTModels/midst_models/single_table_TabDDPM/tabddpm_test",
        "exp_name": "train_1",
        "workspace_dir": "/home/vidit/Desktop/SaTML/MIDSTModels/midst_models/single_table_TabDDPM/tabddpm_test",
        "sample_prefix": "",
        "test_data_dir": "/home/vidit/Desktop/SaTML/MIDSTModels/midst_models/single_table_TabDDPM/tabddpm_test"
    },
    "clustering": {
        "parent_scale": 1.0,
        "num_clusters": 50,
        "clustering_method": "both"
    },
    "diffusion": {
        "d_layers": [
            512,
            1024,
            1024,
            1024,
            1024,
            512
        ],
        "dropout": 0.0,
        "num_timesteps": 2000,
        "model_type": "mlp",
        "iterations": 200000,
        "batch_size": 4096,
        "lr": 0.0006,
        "gaussian_loss_type": "mse",
        "weight_decay": 1e-05,
        "scheduler": "cosine"
    },
    "classifier": {
        "d_laye

In [17]:
relation_order = [[None,'trans']]
models = clava_load_pretrained(relation_order,save_dir)

None -> trans checkpoint found, loading...


In [18]:
model = models[None,'trans']['diffusion']

In [19]:
device = "cuda"

In [20]:
model.to(device)

GaussianMultinomialDiffusion(
  (_denoise_fn): MLPDiffusion(
    (mlp): MLP(
      (blocks): ModuleList(
        (0): Block(
          (linear): Linear(in_features=128, out_features=512, bias=True)
          (activation): ReLU()
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (1): Block(
          (linear): Linear(in_features=512, out_features=1024, bias=True)
          (activation): ReLU()
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (2): Block(
          (linear): Linear(in_features=1024, out_features=1024, bias=True)
          (activation): ReLU()
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (3): Block(
          (linear): Linear(in_features=1024, out_features=1024, bias=True)
          (activation): ReLU()
          (dropout): Dropout(p=0.0, inplace=False)
        )
        (4): Block(
          (linear): Linear(in_features=1024, out_features=1024, bias=True)
          (activation): ReLU()
          (dropout): Dro

## Creating funcitons for GSA

### writing code to implement gsa model
#### Notes
- Can we augment gsa1 with pca so we reduce the dimensionality of the gradients so that we dont have to compromise with the accuracy as well as compute?
- 

In [21]:
tabddpm = model

In [10]:
def gsa1_attack(
    model: GaussianMultinomialDiffusion, 
    x0: torch.Tensor, 
    num_timesteps: int = 10
)-> List[float]:
    
    model.eval()
    timesteps = torch.linspace(0, model.num_timesteps-1, num_timesteps).long()
    losses = []
    
    # Compute losses at sampled timesteps
    for t in timesteps:
        #xt, epsilon = forward_diffusion(x0, t, model)
        x_t = model.gaussian_q_sample(x_start=x0, t=torch.tensor([t]))
        loss = compute_loss(model, xt, t, epsilon)
        losses.append(loss)
    
    # Average losses and compute gradients
    avg_loss = torch.mean(torch.stack(losses))
    model.zero_grad()
    avg_loss.backward()
    
    # Aggregate gradients (L2 norm per layer)
    gradients = [param.grad.detach().norm().item() for param in model.parameters()]
    return gradients

In [11]:
# this is just a code you wrote to check the denoising output of the tabddpm model; remove it later
x1 = torch.tensor([1605,2,4,3900.0,41832.1,1,0,0]).to(device)
x2 = torch.tensor([2006,2,4,14.6,13224.4,6,0,0]).to(device)
x = torch.tensor([x1,x2]).to(device)
xt = tabddpm.gaussian_q_sample(x_start = x, t = torch.tensor([1995]).to(device))
print(xt)
y= {'y': 0}
losses = tabddpm.mixed_loss(x,y)

ValueError: only one element tensors can be converted to Python scalars

In [16]:
x1 = torch.tensor([1605,2,4,3900.0,41832.1,1,0,0]).to(device)
x2 = torch.tensor([2006,2,4,14.6,13224.4,6,0,0]).to(device)
x = torch.tensor([[1605,2,4,3900.0,41832.1,1,0,0],[2006,2,4,14.6,13224.4,6,0,0]]).to(device)
y = {'y':[0,1]}
losses = tabddpm.mixed_loss(x)
print(losses)

TypeError: mixed_loss() missing 1 required positional argument: 'out_dict'

In [ ]:
def gsa2_attack(
    model: GaussianMultinomialDiffusion, 
    x0: torch.Tensor, 
    num_timesteps: int = 10
) -> List[float]:
    model.eval()
    timesteps = torch.linspace(0, model.T-1, num_timesteps).long()
    gradients_accum = None
    
    for t in timesteps:
        xt, epsilon = model.gaussian_q_sample(x0, t, model)
        loss = compute_loss(model, xt, t, epsilon)
        
        # Compute gradients for this timestep
        model.zero_grad()
        loss.backward()
        current_gradients = [param.grad.detach().clone() for param in model.parameters()]
        
        # Accumulate gradients
        if gradients_accum is None:
            gradients_accum = current_gradients
        else:
            gradients_accum = [g_prev + g_curr for g_prev, g_curr in zip(gradients_accum, current_gradients)]
    
    # Average gradients across timesteps
    averaged_gradients = [g / num_timesteps for g in gradients_accum]
    
    # Aggregate gradients (L2 norm per layer)
    aggregated = [g.norm().item() for g in averaged_gradients]
    return aggregated

In [ ]:
# Load a pre-trained diffusion model (example)
x0 = torch.randn(1, 3, 32, 32)  # Example input (CIFAR-10 image)

# Extract features using GSA1 and GSA2
gsa1_features = gsa1_attack(model, x0, num_timesteps=10)
gsa2_features = gsa2_attack(model, x0, num_timesteps=10)

print("GSA1 Features:", gsa1_features)
print("GSA2 Features:", gsa2_features)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# Example dataset (member=1, non-member=0)
X_train = np.array([gsa1_features, gsa2_features])  # Replace with actual features
y_train = np.array([1, 0])

# Train a classifier (e.g., Random Forest)
clf = RandomForestClassifier()
clf.fit(X_train, y_train)